In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [2]:
import sys
sys.path.append("..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel
from utilities.dfdb import DFDB

In [3]:
import numpy as np
import pandas as pd
import os

import lightgbm as lgb

import optuna

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook, tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold

import eli5
from eli5.sklearn import PermutationImportance

from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from IPython.display import HTML
import json
import altair as alt

import copy

import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

alt.renderers.enable('notebook')

Using TensorFlow backend.


RendererRegistry.enable('notebook')

In [3]:
def mem_check(scope):
    list_ = []
    for var_name in scope:
        if not var_name.startswith("_"):
            mem_usage = sys.getsizeof(eval(var_name))/1e6
            list_.append({'variable_name':var_name, 'memory_usage(M)':mem_usage, 'address':id(var_name), 'refcount':sys.getrefcount(eval(var_name))})
    df_ = pd.DataFrame(list_)
    return df_[df_['memory_usage(M)']>1]

In [63]:
mem_check(dir())

,address,memory_usage(M),refcount,variable_name
24,140573361664176,40.088776000000003,2,df_submit
25,140576074964640,24231.501509999998234,9,df_test
26,140573361770160,80.177447999999998,5,df_test_pred
27,140573426287600,14663.846779999999853,2,df_train
28,140573363496048,283.340044999999975,2,df_trial
29,140573361769968,111.795631999999998,5,df_valid_pred
53,140576656094896,40.088776000000003,2,submission


In [64]:
gc.collect()

984

In [4]:
df_train = pd.read_pickle('../data/feats/feats_train.pkl')
df_test = pd.read_pickle('../data/feats/feats_test.pkl')

In [25]:
submission = pd.read_csv('../data/input/sample_submission.csv')

In [109]:
submission.head()

,id,scalar_coupling_constant
0,4658147,0
1,4658148,0
2,4658149,0
3,4658150,0
4,4658151,0


In [28]:
submission.id.min(), submission.id.max()

(4658147, 7163688)

In [30]:
df_test.id.min(), df_test.id.max(), 

(4658147, 7163688)

In [15]:
set(df_test.columns.tolist())-set(df_train.columns.tolist())

{'atom_0',
 'atom_1',
 'atom_x',
 'atom_y',
 'bond_type',
 'eem2015ba_formal_charges_entropy_',
 'eem2015bm_formal_charges_entropy_',
 'eem2015bn_formal_charges_entropy_',
 'eem2015ha_formal_charges_entropy_',
 'eem2015hm_formal_charges_entropy_',
 'eem2015hn_formal_charges_entropy_',
 'eem_formal_charges_entropy_',
 'gasteiger_formal_charges_entropy_',
 'id',
 'mmff94_formal_charges_entropy_',
 'molecule_name',
 'qeq_formal_charges_entropy_',
 'qtpie_formal_charges_entropy_',
 'type',
 'type_0',
 'type_1'}

In [12]:
len(set(df_test.columns.tolist())),len(df_test.columns.tolist()),len(df_test.columns.unique()), 

(1211, 1211, 1211)

In [13]:
'index' in df_test.columns

True

In [32]:
# df_test = df_test.groupby(level=0, axis=1).mean()

In [8]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [7]:
# df_train = reduce_mem_usage(df_train)

Mem. usage decreased to 10568.36 Mb (24.4% reduction)


In [8]:
# df_train.shape[0]*.6

2794888.1999999997

In [54]:
def _str2class(s):
    if s in globals() and isinstance(globals()[s], type):
            return globals()[s]
    if isinstance(eval(s), type):
        return eval(s)
    if callable(eval(s)):
        return eval(s)
    return None

def sk_process(df_train, param, message, df_test=None, trial=None, is_output_feature_importance=False, trial_level=0):

    columns = param['columns']

    assert 'y' in df_train.columns.tolist(), 'y is not in df_train'
    assert 'index' in df_train.columns.tolist(), 'index is not in df_train'
    assert 'index' not in param['columns'], 'index is in features'
    assert 'y' not in param['columns'], 'y is in features'
    assert 'label' not in param['columns'], 'label is in features'
    assert 'group' not in param['columns'], 'group is in features'
    assert (type(trial) == list) | (trial == None), 'trial is neither list nor none'
    assert len(columns) != 0, 'columns size is 0'

    df_test_pred = None
    if type(df_test) == pd.DataFrame:
        assert 'index' in df_test.columns.tolist(), 'index is not in df_test'
        df_test_pred = pd.concat([df_test_pred, df_test[['index']]], axis=1)
        
    CV = _str2class(param['cv']['cls'])
    MODEL = _str2class(param['model']['cls'])
    if 'scaler' in param:
        SCALER = _str2class(param['scaler']['cls'])
    metric = _str2class(param['metric'])
    
    history = []
    df_valid_pred = pd.DataFrame()
    df_feature_importances_i_list = []

    # StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
    if 'splits' in param['cv']:
        splits = param['cv']['splits']
    else:
        cv = CV(**param['cv']['init'])
        if param['cv']['cls'] == 'StratifiedKFold':
            assert 'label' in df_train.columns.tolist(), 'label is not in df_train'
            splits = list(cv.split(df_train, df_train['label']))
        elif param['cv']['cls'] == 'GroupKFold':
            assert 'group' in df_train.columns.tolist(), 'group is not in df_train'
            splits = list(cv.split(df_train, groups=df_train['group']))
        else:
            splits = list(cv.split(df_train))

    for fold_n, (train_index, valid_index) in enumerate(splits):

        X_train, X_valid = df_train[columns].values[train_index, :], df_train[columns].values[valid_index, :]
        y_train, y_valid = df_train['y'].values[train_index], df_train['y'].values[valid_index]

        if 'scaler' in param:
            scaler = SCALER(**param['scaler']['init'])
            X_train = scaler.fit_transform(X_train)
            X_valid = scaler.transform(X_valid)

        model = MODEL(**param['model']['init'])
        model.fit(X_train, y_train, **param['model']['fit'])

        y_valid_pred = model.predict(X_valid)
        y_train_pred = model.predict(X_train)

        original_index = df_train['index'].values[valid_index]
        df_valid_pred_i = pd.DataFrame({'index': original_index, 'predict': y_valid_pred, 'fold_n': np.zeros(y_valid_pred.shape[0]) + fold_n})
        df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)

        if is_output_feature_importance:
            df_feature_importances_i = pd.DataFrame({'feature': columns, 'model_weight': model.feature_importances_})
            df_feature_importances_i = df_feature_importances_i.sort_values(by=['feature'])
            df_feature_importances_i = df_feature_importances_i.reset_index(drop=True)
            perm = PermutationImportance(model, random_state=42).fit(X_valid, y_valid)
            df_feature_importances_i2 = eli5.explain_weights_dfs(perm, feature_names=columns, top=len(columns))['feature_importances']
            df_feature_importances_i2 = df_feature_importances_i2.sort_values(by=['feature'])
            df_feature_importances_i2 = df_feature_importances_i2.reset_index(drop=True)
            df_feature_importances_i = pd.merge(df_feature_importances_i, df_feature_importances_i2, on='feature')
            df_feature_importances_i_list.append(df_feature_importances_i)

        if type(df_test) == pd.DataFrame:
            X_test = df_test[columns].values
            if 'scaler' in param:
                X_test = scaler.transform(X_test)
            y_test_pred = model.predict(X_test)
            df_test_pred_i = pd.DataFrame({fold_n: y_test_pred})
            df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=1)
        
        history.append({'fold_n': fold_n, 'train': metric(y_train, y_train_pred), 'valid': metric(y_valid, y_valid_pred)})

    df_his = pd.DataFrame(history)

    df_feature_importances = None
    if is_output_feature_importance:
        df_feature_importances = df_feature_importances_i_list[0]
        for idx, df_feature_importances_i in enumerate(df_feature_importances_i_list[1:]):
            df_feature_importances = pd.merge(df_feature_importances, df_feature_importances_i, on='feature', suffixes=('', idx + 1))

    df_valid_pred = df_valid_pred.sort_values(by=['index'])
    df_valid_pred = df_valid_pred.reset_index(drop=True)

    if type(df_test) == pd.DataFrame:
        df_test_pred = df_test_pred.sort_values(by=['index'])
        df_test_pred = df_test_pred.reset_index(drop=True)

    if type(trial) == list:
        datetime_ = datetime.datetime.now()
        val_metric_mean = np.mean(df_his.valid)
        val_metric_std = np.std(df_his.valid)
        train_metric_mean = np.mean(df_his.train)
        train_metric_std = np.std(df_his.train)

        trial_i_d_ = {'datetime': datetime_, 'message': message, 'val_metric_mean': val_metric_mean,
                  'train_metric_mean': train_metric_mean, 'val_metric_std': val_metric_std, 'train_metric_std': train_metric_std,
                  'trn_val_metric_diff': val_metric_mean - train_metric_mean,
                  'df_feature_importances': df_feature_importances,'param': param.copy(),
                  'nfeatures': len(columns)}
        if trial_level > 0:
            trial_i_d_ = {'df_his': df_his, 'df_valid_pred': df_valid_pred, 'df_test_pred': df_test_pred, **trial_i_d_}
        trial.append(trial_i_d_)

    return df_his, df_feature_importances, df_valid_pred, df_test_pred

In [10]:
columns = df_train.drop(columns=['index','y']).columns.tolist()

In [11]:
param = {
    'columns': columns,
    'cv': {
        'cls': 'KFold',
        'init':{
            'n_splits': 3,
            'shuffle': True,
            'random_state': 42,
        },
    },
    'scaler': {
        'cls': 'StandardScaler',
        'init':{},
        'fit':{},
    },
    'model': {
        'cls': 'lgb.LGBMRegressor',
        'init': {
        },
        'fit': {
        },
    },
    'metric':'mean_absolute_error',
}

In [12]:
# trial = []
df_his, df_feature_importances, df_valid_pred, df_test_pred = sk_process(df_train.loc[:1000000], param, 'find best cv', df_test=df_train.loc[1000000:2000000], trial=trial, is_output_feature_importance=True, trial_level=0)

In [30]:
sorted_columns = evaluate(df_feature_importances, 'average_permutation_weight')

In [9]:
sorted_columns=['L2dist',
 'dist_to_type_mean',
 'atom_index_1_bonds2_n_bonds',
 'molecule_atom_index_1_dist_min',
 'dist',
 'cos_0',
 'distance',
 'atom_index_1_bonds2_bond_lengths_std',
 'atom_index_1_bonds2_bond_lengths_mean',
 'atom_index_0_bonds2_bond_lengths_mean',
 'is_found_bond',
 'molecule_atom_index_0_dist_std_diff',
 'molecule_atom_index_0_dist_std_div',
 'dist_to_type_0_mean',
 'cos_0_1',
 'molecule_atom_index_0_dist_min',
 'atom_1_couples_count',
 'dist_to_type_1_mean',
 'int_',
 'molecule_atom_1_dist_std_diff',
 'molecule_atom_1_dist_min',
 'molecule_atom_index_0_dist_min_div',
 'molecule_atom_index_0_dist_min_diff',
 'cos_1',
 'molecule_atom_index_1_dist_std_diff',
 'molecule_couples',
 'molecule_type_dist_std',
 'molecule_atom_index_0_dist_max',
 'distance_closest_1',
 'molecule_atom_index_0_dist_max_div',
 'atom_index_1_mean_cycle_atom',
 'molecule_atom_index_0_dist_std',
 'molecule_atom_1_dist_min_diff',
 'molecule_atom_1_dist_min_div',
 'molecule_type_dist_max',
 'molecule_atom_index_0_dist_max_diff',
 'bonds3_bond_angle_plane',
 'molecule_atom_index_0_dist_mean',
 'molecule_atom_index_0_dist_mean_diff',
 'molecule_atom_index_1_dist_max',
 'molecule_atom_index_1_dist_std_div',
 'atom_0_couples_count',
 'atom_index_1_n_cycle',
 'molecule_atom_index_0_dist_mean_div',
 'molecule_type_0_dist_std_diff',
 'molecule_atom_index_1_dist_mean_diff',
 'molecule_atom_index_1_dist_mean',
 'molecule_type_0_dist_std',
 'molecule_type_dist_min',
 'bonds3_bond_angle_axis',
 'atom_index_closest_0',
 'molecule_atom_index_1_dist_max_diff',
 'molecule_atom_index_1_dist_mean_div',
 'molecule_atom_index_1_dist_min_diff',
 'molecule_type_dist_mean_div',
 'molecule_atom_index_1_dist_min_div',
 'distance_closest_0',
 'bonds3_flatness',
 'molecule_atom_index_1_dist_std',
 'gasteiger_partical_charges_quantile05_',
 'eem2015hm_partical_charges_abs_quantile05_',
 'eem2015bm_partical_charges_no_zero_crossing_',
 'y_1',
 'atom_index_0_bonds2_bonds_mean',
 'molecule_dist_max',
 'mmff94_partical_charges_min_',
 'eem_partical_charges_quantile75_',
 'eem2015hm_partical_charges_entropy_',
 'molecule_atom_1_dist_std',
 'molecule_atom_index_1_dist_max_div',
 'dist_y',
 'dy',
 'mmff94_partical_charges_max_',
 'mmff94_partical_charges_quantile75_',
 'qeq_partical_charges_quantile25_',
 'mmff94_partical_charges_abs_quantile01_',
 'eem2015hn_partical_charges_abs_quantile95_',
 'mmff94_partical_charges_peak_peak_amp_',
 'molecule_type_dist_mean_diff',
 'atom_index_1',
 'gasteiger_partical_charges_mean_',
 'atom_index_1_bonds2_bonds_std',
 'gasteiger_partical_charges_min_',
 'molecule_type_dist_std_diff',
 'molecule_atom_1_dist_mean',
 'eem2015hn_partical_charges_median_',
 'eem2015hm_partical_charges_median_',
 'mmff94_partical_charges_skewness_',
 'qeq_partical_charges_median_',
 'molecule_type_dist_mean',
 'molecule_dist_min',
 'eem2015hm_partical_charges_skewness_',
 'eem2015ha_partical_charges_quantile75_',
 'eem_partical_charges_skewness_',
 'qeq_partical_charges_quantile99_',
 'eem2015bm_partical_charges_quantile75_',
 'qtpie_partical_charges_min_',
 'molecule_atom_index_0_y_1_mean',
 'molecule_atom_index_0_y_1_std',
 'qtpie_partical_charges_median_',
 'eem_partical_charges_min_',
 'gasteiger_partical_charges_quantile25_',
 'eem_partical_charges_median_',
 'eem2015bm_partical_charges_abs_quantile01_',
 'eem_partical_charges_abs_quantile25_',
 'eem2015hm_partical_charges_abs_quantile01_',
 'qeq_partical_charges_max_',
 'eem_partical_charges_quantile05_',
 'gasteiger_partical_charges_abs_quantile75_',
 'qeq_partical_charges_min_',
 'dx',
 'mmff94_partical_charges_entropy_',
 'qeq_partical_charges_abs_quantile25_',
 'mmff94_partical_charges_abs_quantile25_',
 'molecule_atom_index_0_y_1_max_diff',
 'mmff94_partical_charges_quantile05_',
 'eem2015ha_partical_charges_quantile25_',
 'eem2015hm_partical_charges_quantile75_',
 'atom_index_closest_1',
 'eem2015bn_partical_charges_abs_quantile05_',
 'bonds3_size',
 'molecule_atom_index_0_y_1_mean_div',
 'eem2015bm_partical_charges_skewness_',
 'x_0',
 'eem2015hn_partical_charges_abs_quantile01_',
 'eem2015bn_partical_charges_quantile75_',
 'gasteiger_partical_charges_abs_quantile95_',
 'gasteiger_partical_charges_clearance_factor_',
 'gasteiger_partical_charges_max_',
 'eem2015hm_partical_charges_quantile25_',
 'eem_partical_charges_kurtosis_',
 'eem2015bn_partical_charges_no_zero_crossing_',
 'eem2015bn_dipole_moment_x',
 'qtpie_partical_charges_skewness_',
 'molecule_atom_index_0_y_1_mean_diff',
 'mmff94_partical_charges_quantile25_',
 'mmff94_partical_charges_quantile99_',
 'molecule_atom_index_0_z_1_std',
 'eem2015hn_partical_charges_quantile05_',
 'eem2015hn_partical_charges_no_zero_crossing_',
 'mmff94_dipole_moment_abs_quantile95_',
 'eem2015bn_partical_charges_form_factor_',
 'eem2015ba_partical_charges_max_',
 'eem_partical_charges_no_zero_crossing_',
 'mmff94_partical_charges_clearance_factor_',
 'eem2015ba_partical_charges_quantile25_',
 'eem2015bn_dipole_moment_y',
 'gasteiger_partical_charges_abs_quantile99_',
 'gasteiger_partical_charges_quantile75_',
 'qeq_partical_charges_form_factor_',
 'eem2015hm_partical_charges_abs_quantile99_',
 'eem2015bn_partical_charges_quantile01_',
 'eem2015ba_partical_charges_abs_quantile25_',
 'eem2015bm_dipole_moment_x',
 'atom_index_1_bonds2_bonds_mean',
 'eem2015ha_partical_charges_crest_factor_',
 'gasteiger_dipole_moment_rms_',
 'qeq_partical_charges_skewness_',
 'gasteiger_dipole_moment_smoothness_',
 'eem2015hm_partical_charges_min_',
 'eem2015ba_partical_charges_skewness_',
 'eem2015ba_partical_charges_entropy_',
 'eem2015bm_partical_charges_median_',
 'eem2015bn_partical_charges_entropy_',
 'mmff94_dipole_moment_abs_quantile75_',
 'eem2015bm_partical_charges_abs_quantile05_',
 'atom_index_0',
 'eem2015bn_partical_charges_quantile05_',
 'eem_partical_charges_abs_quantile01_',
 'eem2015hm_partical_charges_abs_quantile75_',
 'eem2015ba_partical_charges_quantile75_',
 'mmff94_partical_charges_abs_quantile95_',
 'qtpie_partical_charges_abs_quantile99_',
 'eem_partical_charges_quantile01_',
 'molecule_atom_index_0_y_1_max',
 'eem2015hm_partical_charges_max_',
 'eem2015ba_partical_charges_impluse_indicator_',
 'eem2015hn_partical_charges_max_',
 'eem2015hn_partical_charges_skewness_',
 'eem2015hm_partical_charges_no_zero_crossing_',
 'eem2015ha_partical_charges_abs_quantile75_',
 'dist_x',
 'eem2015ha_partical_charges_peak_peak_amp_',
 'eem2015ba_partical_charges_quantile99_',
 'eem_dipole_moment_y',
 'mmff94_partical_charges_median_',
 'eem2015hm_partical_charges_quantile99_',
 'eem2015bn_partical_charges_variance_',
 'eem2015ha_partical_charges_quantile99_',
 'qeq_partical_charges_quantile01_',
 'eem2015bm_partical_charges_quantile05_',
 'eem2015bm_partical_charges_abs_quantile25_',
 'qeq_dipole_moment_y',
 'eem2015hn_partical_charges_abs_quantile25_',
 'eem2015bn_partical_charges_skewness_',
 'gasteiger_partical_charges_entropy_',
 'y_closest_0',
 'eem2015ha_partical_charges_kurtosis_',
 'eem_partical_charges_quantile25_',
 'eem2015ba_partical_charges_abs_quantile95_']

In [48]:
param = {
    'columns': sorted_columns,
    'cv': {
        'cls': 'KFold',
        'init':{
            'n_splits': 3,
            'shuffle': True,
            'random_state': 42,
        },
    },
    'scaler': {
        'cls': 'StandardScaler',
        'init':{},
        'fit':{},
    },
    'model': {
        'cls': 'lgb.LGBMRegressor',
        'init': {
        },
        'fit': {
        },
    },
    'metric':'mean_absolute_error',
}
select_features_(df_train.loc[:1000000], param, trial, message='start from top 200' , key='average_permutation_weight', nfeats_best=20, nfeats_removed_per_try=20)

In [52]:
df_trial = pd.DataFrame(trial)
df_trial[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff']]

,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff
0,2019-07-03 01:14:18.731972,find best cv,1190,1.483795641064289,1.493590913949344,0.009795272885054
1,2019-07-03 02:16:32.829909,start from top 200,200,1.488183443031910,1.497240349158590,0.009056906126680
2,2019-07-03 02:34:12.383320,start from top 200,180,1.484211488823717,1.493163252889837,0.008951764066121
3,2019-07-03 02:49:55.031210,start from top 200,160,1.486388783559104,1.496000111281279,0.009611327722175
4,2019-07-03 03:03:18.001571,start from top 200,140,1.487021579971061,1.496610389669181,0.009588809698120
5,2019-07-03 03:14:16.082958,start from top 200,120,1.487358208542042,1.496738611041003,0.009380402498962
6,2019-07-03 03:23:22.017322,start from top 200,100,1.483593274620089,1.493200103913104,0.009606829293015
7,2019-07-03 03:30:45.631368,start from top 200,80,1.487285846884541,1.495363906570139,0.008078059685598
8,2019-07-03 03:35:55.082168,start from top 200,60,1.478301581046608,1.486511932918193,0.008210351871586
9,2019-07-03 03:39:28.035011,start from top 200,40,1.481931678005836,1.489040018412062,0.007108340406227


In [62]:
param = {
    'columns': df_trial.loc[17]['param']['columns'],
    'cv': {
        'cls': 'KFold',
        'init':{
            'n_splits': 3,
            'shuffle': True,
            'random_state': 42,
        },
    },
    'scaler': {
        'cls': 'StandardScaler',
        'init':{},
        'fit':{},
    },
    'model': {
        'cls': 'lgb.LGBMRegressor',
        'init': {
        },
        'fit': {
        },
    },
    'metric':'mean_absolute_error',
}
width_frist_rfe(df_train.loc[:1000000], param, trial, score=df_trial.loc[17]['val_metric_mean'], message='width_frist_rfe 60')

In [68]:
# df_trial = pd.DataFrame(trial)
df_trial[df_trial['message']=='width_frist_rfe 60'].sort_values(by=['val_metric_mean'], ascending=True)[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff']].head()

,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff
246,2019-07-03 12:12:52.032327,width_frist_rfe 60,58,1.474902001337016,1.483090482016729,0.008188480679713
287,2019-07-03 12:29:37.244135,width_frist_rfe 60,57,1.474902001337015,1.483115968597858,0.008213967260843
236,2019-07-03 12:08:45.906387,width_frist_rfe 60,58,1.475869042965285,1.483532386075154,0.007663343109869
216,2019-07-03 12:00:34.818899,width_frist_rfe 60,58,1.475710614968435,1.484336602810757,0.008625987842322
230,2019-07-03 12:06:19.971281,width_frist_rfe 60,58,1.475939306592114,1.484665681630609,0.008726375038494


In [75]:
def objective(trial):
        
    learning_rate = trial.suggest_uniform('learning_rate', .01, .5)
    feature_fraction = trial.suggest_uniform('feature_fraction', .6, 1)
    bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.6, 1)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 200, 800)
    lambda_l1 = trial.suggest_loguniform('lambda_l1', 1e-6, 1e2)
    lambda_l2 = trial.suggest_loguniform('lambda_l2', 1e-6, 1e2)
    max_bin = trial.suggest_int('max_bin', 10, 100)
    num_leaves = trial.suggest_int('num_leaves', 4, 64)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':df_trial.loc[246]['param']['columns'],
        'cv': {
            'cls': 'KFold',
            'init':{
                'n_splits': 3,
                'shuffle': True,
                'random_state': 42,
            },
        },
        'scaler': {
            'cls': 'StandardScaler',
            'init':{},
            'fit':{},
        },
        'model': {
            'cls': 'lgb.LGBMRegressor',
            'init': {
                'learning_rate':learning_rate,
                'feature_fraction':feature_fraction,
                'bagging_fraction':bagging_fraction,
                'min_data_in_leaf':min_data_in_leaf,
                'lambda_l1':lambda_l1,
                'lambda_l2':lambda_l2,
                'max_bin':max_bin,
                'num_leaves':num_leaves,
                'random_state':random_state,
                'n_jobs':16
            },
            'fit': {
            },
        },
        'metric':'mean_absolute_error',
    }
    
    df_his, df_feature_importances, df_valid_pred, df_test_pred =  sk_process(df_train.loc[:1000000], args, 'tune 246', trial=mytrial, is_output_feature_importance=False, trial_level=0)
    val_metric_mean = np.mean(df_his.valid)
    return val_metric_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-07-03 13:31:59,756] Finished a trial resulted in value: 1.1944945214735128. Current best value is 1.1944945214735128 with parameters: {'learning_rate': 0.4967370487508111, 'feature_fraction': 0.9122357178136471, 'bagging_fraction': 0.721243566739601, 'min_data_in_leaf': 391, 'lambda_l1': 28.42659569995838, 'lambda_l2': 1.7467638646215318e-06, 'max_bin': 36, 'num_leaves': 45, 'random_state': 9499}.
[I 2019-07-03 13:32:22,600] Finished a trial resulted in value: 1.3006658205807857. Current best value is 1.1944945214735128 with parameters: {'learning_rate': 0.4967370487508111, 'feature_fraction': 0.9122357178136471, 'bagging_fraction': 0.721243566739601, 'min_data_in_leaf': 391, 'lambda_l1': 28.42659569995838, 'lambda_l2': 1.7467638646215318e-06, 'max_bin': 36, 'num_leaves': 45, 'random_state': 9499}.
[I 2019-07-03 13:32:47,413] Finished a trial resulted in value: 6.817523026793881. Current best value is 1.1944945214735128 with parameters: {'learning_rate': 0.4967370487508111, 'fe

[I 2019-07-03 13:46:39,014] Finished a trial resulted in value: 1.2235665131131601. Current best value is 1.1322097548663077 with parameters: {'learning_rate': 0.49406232537237543, 'feature_fraction': 0.8073271361540657, 'bagging_fraction': 0.6081145066019186, 'min_data_in_leaf': 358, 'lambda_l1': 0.5150125406240468, 'lambda_l2': 0.04017862928714584, 'max_bin': 59, 'num_leaves': 64, 'random_state': 7803}.
[I 2019-07-03 13:46:59,801] Finished a trial resulted in value: 1.3766573292059643. Current best value is 1.1322097548663077 with parameters: {'learning_rate': 0.49406232537237543, 'feature_fraction': 0.8073271361540657, 'bagging_fraction': 0.6081145066019186, 'min_data_in_leaf': 358, 'lambda_l1': 0.5150125406240468, 'lambda_l2': 0.04017862928714584, 'max_bin': 59, 'num_leaves': 64, 'random_state': 7803}.
[I 2019-07-03 13:47:21,498] Finished a trial resulted in value: 1.1445298990144142. Current best value is 1.1322097548663077 with parameters: {'learning_rate': 0.49406232537237543, '

[I 2019-07-03 14:01:19,544] Finished a trial resulted in value: 1.333095540465579. Current best value is 1.1305642809889787 with parameters: {'learning_rate': 0.2914021475905548, 'feature_fraction': 0.8464111694628749, 'bagging_fraction': 0.8715487818254323, 'min_data_in_leaf': 215, 'lambda_l1': 0.35479660685447506, 'lambda_l2': 5.761603776936107e-05, 'max_bin': 69, 'num_leaves': 64, 'random_state': 3898}.
[I 2019-07-03 14:01:42,516] Finished a trial resulted in value: 1.1659744149103852. Current best value is 1.1305642809889787 with parameters: {'learning_rate': 0.2914021475905548, 'feature_fraction': 0.8464111694628749, 'bagging_fraction': 0.8715487818254323, 'min_data_in_leaf': 215, 'lambda_l1': 0.35479660685447506, 'lambda_l2': 5.761603776936107e-05, 'max_bin': 69, 'num_leaves': 64, 'random_state': 3898}.
[I 2019-07-03 14:02:05,772] Finished a trial resulted in value: 1.29259366814067. Current best value is 1.1305642809889787 with parameters: {'learning_rate': 0.2914021475905548, '

[I 2019-07-03 14:15:35,823] Finished a trial resulted in value: 1.1463450267089141. Current best value is 1.1305642809889787 with parameters: {'learning_rate': 0.2914021475905548, 'feature_fraction': 0.8464111694628749, 'bagging_fraction': 0.8715487818254323, 'min_data_in_leaf': 215, 'lambda_l1': 0.35479660685447506, 'lambda_l2': 5.761603776936107e-05, 'max_bin': 69, 'num_leaves': 64, 'random_state': 3898}.
[I 2019-07-03 14:15:58,698] Finished a trial resulted in value: 1.1692665742657455. Current best value is 1.1305642809889787 with parameters: {'learning_rate': 0.2914021475905548, 'feature_fraction': 0.8464111694628749, 'bagging_fraction': 0.8715487818254323, 'min_data_in_leaf': 215, 'lambda_l1': 0.35479660685447506, 'lambda_l2': 5.761603776936107e-05, 'max_bin': 69, 'num_leaves': 64, 'random_state': 3898}.
[I 2019-07-03 14:16:20,540] Finished a trial resulted in value: 1.3281455339079125. Current best value is 1.1305642809889787 with parameters: {'learning_rate': 0.2914021475905548

[I 2019-07-03 14:30:25,320] Finished a trial resulted in value: 1.1280761107528405. Current best value is 1.1154625936983689 with parameters: {'learning_rate': 0.4997988762193408, 'feature_fraction': 0.7854653555135787, 'bagging_fraction': 0.9930502870818868, 'min_data_in_leaf': 200, 'lambda_l1': 81.78442876857926, 'lambda_l2': 42.17658760982638, 'max_bin': 86, 'num_leaves': 64, 'random_state': 9950}.
[I 2019-07-03 14:30:50,483] Finished a trial resulted in value: 1.1298599365836381. Current best value is 1.1154625936983689 with parameters: {'learning_rate': 0.4997988762193408, 'feature_fraction': 0.7854653555135787, 'bagging_fraction': 0.9930502870818868, 'min_data_in_leaf': 200, 'lambda_l1': 81.78442876857926, 'lambda_l2': 42.17658760982638, 'max_bin': 86, 'num_leaves': 64, 'random_state': 9950}.
[I 2019-07-03 14:31:13,394] Finished a trial resulted in value: 1.1222446296133126. Current best value is 1.1154625936983689 with parameters: {'learning_rate': 0.4997988762193408, 'feature_f

[I 2019-07-03 14:45:19,708] Finished a trial resulted in value: 1.1579907874634074. Current best value is 1.1012805145464435 with parameters: {'learning_rate': 0.487977189229769, 'feature_fraction': 0.7708485101270728, 'bagging_fraction': 0.9888018379624192, 'min_data_in_leaf': 205, 'lambda_l1': 96.68773760389033, 'lambda_l2': 5.775688925032861, 'max_bin': 94, 'num_leaves': 64, 'random_state': 9879}.
[I 2019-07-03 14:45:42,598] Finished a trial resulted in value: 1.2016384817923864. Current best value is 1.1012805145464435 with parameters: {'learning_rate': 0.487977189229769, 'feature_fraction': 0.7708485101270728, 'bagging_fraction': 0.9888018379624192, 'min_data_in_leaf': 205, 'lambda_l1': 96.68773760389033, 'lambda_l2': 5.775688925032861, 'max_bin': 94, 'num_leaves': 64, 'random_state': 9879}.
[I 2019-07-03 14:46:03,958] Finished a trial resulted in value: 1.2895816674639626. Current best value is 1.1012805145464435 with parameters: {'learning_rate': 0.487977189229769, 'feature_frac

In [92]:
#df_trial = pd.DataFrame(mytrial)#&(df_trial['trn_val_metric_diff']<.02)
df_trial[(df_trial['message']=='tune 246')].sort_values(by=['val_metric_mean'], ascending=True)[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff']].head(20)

,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff
503,2019-07-03 14:34:14.065151,tune 246,58,1.062459726764430,1.101280514546443,0.038820787782013
509,2019-07-03 14:36:26.095557,tune 246,58,1.072569193940650,1.112513948064277,0.039944754123626
492,2019-07-03 14:30:02.137193,tune 246,58,1.077164088431871,1.115462593698369,0.038298505266497
537,2019-07-03 14:46:49.545059,tune 246,58,1.078365053588329,1.115548545102693,0.037183491514364
501,2019-07-03 14:33:25.067866,tune 246,58,1.072777641307315,1.116386498188080,0.043608856880764
491,2019-07-03 14:29:38.679522,tune 246,58,1.080184033605193,1.116587187526388,0.036403153921194
485,2019-07-03 14:27:21.079614,tune 246,58,1.083877350161698,1.118683911890783,0.034806561729085
531,2019-07-03 14:44:34.824704,tune 246,58,1.083667546557525,1.121148634810648,0.037481088253124
513,2019-07-03 14:37:55.911769,tune 246,58,1.079211743586685,1.121504146008850,0.042292402422165
495,2019-07-03 14:31:13.370871,tune 246,58,1.083653627886729,1.122244629613313,0.038591001726584


In [4]:
db = DFDB('../data/trial/lgbm_trial.pkl')

In [5]:
df_trial = db.select()

In [19]:
df_trial[df_trial['message'] == 'tune 246'].sort_values(by=['val_metric_mean']).head(20)[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff']].head()

,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff
503,2019-07-03 14:34:14.065151,tune 246,58,1.062459726764430,1.101280514546443,0.038820787782013
509,2019-07-03 14:36:26.095557,tune 246,58,1.072569193940650,1.112513948064277,0.039944754123626
492,2019-07-03 14:30:02.137193,tune 246,58,1.077164088431871,1.115462593698369,0.038298505266497
537,2019-07-03 14:46:49.545059,tune 246,58,1.078365053588329,1.115548545102693,0.037183491514364
501,2019-07-03 14:33:25.067866,tune 246,58,1.072777641307315,1.116386498188080,0.043608856880764


In [20]:
param = df_trial.loc[503]['param']

In [61]:
df_test = df_test.drop(columns=['index'])

In [65]:
df_test = df_test.rename(columns={'id':'index'})

In [66]:
db.rollback()

In [67]:
mytrial = []
df_his, df_feature_importances, df_valid_pred, df_test_pred = sk_process(df_train, param, 'try kfold3 1m-trn 1m-val', df_test=df_test, trial=mytrial, is_output_feature_importance=False, trial_level=1)

In [68]:
db.insert(mytrial)
df_trial = db.select()

In [69]:
df_trial[df_trial['message'] == 'try kfold3 1m-trn 1m-val'].sort_values(by=['val_metric_mean'])[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff']]

,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff
539,2019-07-04 06:32:14.616842,try kfold3 1m-trn 1m-val,58,1.130218461804760,1.141611006584587,0.011392544779828
540,2019-07-04 06:42:15.869321,try kfold3 1m-trn 1m-val,58,1.130218461804763,1.141611006584590,0.011392544779827
538,2019-07-04 05:54:38.217844,try kfold3 1m-trn 1m-val,58,1.444710663054743,1.841290190622159,0.396579527567416


In [80]:
db.commit()

In [81]:
idx=540
df_test_pred = df_trial.loc[idx]['df_test_pred']
df_submit = pd.DataFrame()
df_submit['scalar_coupling_constant'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['id'] = df_test_pred['index']
df_submit.to_csv('../data/submit/submission_lgbm_{}.csv'.format(idx), index=False)

In [12]:
m_list = []
for col in sorted_columns:#df_trial.loc[540]['param']['columns']:
    for m in [ "eem", "mmff94", "gasteiger", "qeq", "qtpie",  "eem2015ha", "eem2015hm", "eem2015hn", "eem2015ba", "eem2015bm", "eem2015bn" ]:
        if m in col:
            m_list.append(m)
            break

In [13]:
set(m_list)

{'eem', 'gasteiger', 'mmff94', 'qeq', 'qtpie'}

In [7]:
df_trial.loc[540]['param']['columns']

['dist_to_type_0_mean',
 'molecule_atom_1_dist_min_div',
 'atom_1_couples_count',
 'molecule_atom_1_dist_min_diff',
 'distance',
 'dist_to_type_1_mean',
 'mmff94_partical_charges_min_',
 'molecule_atom_index_1_dist_std',
 'molecule_atom_index_0_dist_max_div',
 'molecule_atom_index_0_dist_std',
 'molecule_atom_index_1_dist_min_diff',
 'atom_index_0_bonds2_bond_lengths_mean',
 'molecule_atom_1_dist_min',
 'molecule_atom_1_dist_std_diff',
 'molecule_atom_index_1_dist_std_diff',
 'molecule_type_0_dist_std_diff',
 'molecule_atom_index_1_dist_max_diff',
 'molecule_type_0_dist_std',
 'atom_index_closest_0',
 'atom_0_couples_count',
 'molecule_atom_index_0_dist_min_div',
 'molecule_atom_index_0_dist_min',
 'cos_1',
 'distance_closest_0',
 'molecule_atom_index_0_dist_mean',
 'molecule_atom_index_1_dist_mean_div',
 'atom_index_1_bonds2_n_bonds',
 'cos_0',
 'is_found_bond',
 'molecule_atom_index_1_dist_max',
 'molecule_atom_index_1_dist_mean_diff',
 'molecule_atom_index_0_dist_std_diff',
 'molecu

In [86]:
original_train_csv = pd.read_csv('../data/input/train.csv')

In [91]:
original_train_csv = original_train_csv[['id', 'molecule_name']]

In [93]:
original_train_csv = original_train_csv.rename(columns={'id':'index'})

In [90]:
df_valid_pred.tail()

,index,predict,fold_n
4658142,4658142,4.091789737704665,1.0
4658143,4658143,-0.136792595529070,1.0
4658144,4658144,8.179541478267220,2.0
4658145,4658145,10.102768289493520,2.0
4658146,4658146,116.541855367460357,1.0


In [94]:
df_valid_pred = pd.merge(df_valid_pred, original_train_csv, on='index')

In [95]:
df_valid_pred.head()

,index,predict,fold_n,molecule_name
0,0,98.809667498132782,0.0,dsgdb9nsd_000001
1,1,-9.803746683755492,1.0,dsgdb9nsd_000001
2,2,-10.075150885314049,0.0,dsgdb9nsd_000001
3,3,-10.119507125308738,0.0,dsgdb9nsd_000001
4,4,94.847075738473606,2.0,dsgdb9nsd_000001


In [100]:
def group_mean_log_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(groups).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()

In [101]:
def group_mean_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(groups).mean()
    return maes.map(lambda x: max(x, floor)).mean()

In [97]:
logmae = group_mean_log_mae(df_train.y, df_valid_pred.predict, df_valid_pred.molecule_name)

In [102]:
mae = group_mean_mae(df_train.y, df_valid_pred.predict, df_valid_pred.molecule_name)

In [106]:
logmae, mae, np.log(1), np.exp(-0.043)

(0.14459448250876014, 1.214430526175789, 0.0, 0.9579113900670306)

In [61]:
def evaluate(df_feature_importances, key='average_model_weight'):
        df_feature_importances['average_permutation_weight'] = df_feature_importances[
            [col for col in df_feature_importances.columns.tolist() if ('weight' in col) & ('model' not in col)]].mean(
            axis=1)
        df_feature_importances['average_model_weight'] = df_feature_importances[
            [col for col in df_feature_importances.columns.tolist() if ('model_weight' in col)]].mean(axis=1)
        df_feature_importances = df_feature_importances.sort_values(by=[key], ascending=False)
        sorted_columns = df_feature_importances.feature.tolist()
        return sorted_columns

def select_features_(df_train, param, trial, message, df_test=None, nfeats_best=10, nfeats_removed_per_try=10, key='average_model_weight'):
    param_i = param.copy()
    while True:
        df_his, df_feature_importances, df_valid_pred, df_test_pred = sk_process(df_train, param_i, df_test=df_test, trial=trial, is_output_feature_importance=True, message=message)
        sorted_columns = evaluate(df_feature_importances, key)
        if (len(sorted_columns) <= nfeats_best)|(len(sorted_columns)-nfeats_removed_per_try<1):
            break
        else:
            param_i['columns'] = sorted_columns[:-nfeats_removed_per_try]
    return

def width_frist_rfe(df_train, param, trial, score, message, df_test=None):

    param_ = copy.deepcopy(param)
    columns_ = param_['columns']
    best_score = score
    best_param = param_
    for col in columns_:
        param_['columns'] = list(set(columns_) - set([col]))
        df_his, df_feature_importances, df_valid_pred, df_test_pred = sk_process(df_train, param_, df_test=df_test, trial=trial, is_output_feature_importance=False, message=message)
        val_mae_mean = np.mean(df_his.valid)
        if val_mae_mean<best_score:
            best_score = val_mae_mean
            best_param = copy.deepcopy(param_)

    if best_score < score:
        width_frist_rfe(df_train, best_param, trial, best_score, message, df_test)

    return

def revert_rfe(df_train, param, sorted_columns, df_test, trial, start_columns, limit=None, remark=None):

    # init cv_score and try only base feature
    selected_columns = copy.deepcopy(start_columns)
    if type(limit) == type(None):
        limit = len(sorted_columns)
    args = copy.deepcopy(param)
    args['columns'] = selected_columns
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=trial, remark=remark)
    val_mae_mean = np.mean(df_his.valid)
    cv_score = val_mae_mean

    # add feature one by one and check cv score change
    for idx,col in enumerate(sorted_columns):
#         if idx in start_column_index:
#             continue
        args = copy.deepcopy(param)
        args['columns'] = list(set(selected_columns + [col]))
        df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=trial, remark=remark)
        val_mae_mean = np.mean(df_his.valid)
        if val_mae_mean < cv_score:
            selected_columns.append(col)
            cv_score = val_mae_mean
        if len(selected_columns) >= limit:
            break

    return selected_columns

def blacklist_merge(df, columns=None, base_correlation_coefficient=.9):

    if type(columns)==type(None):
        columns = df.columns.tolist()
    bcc_ = base_correlation_coefficient
    X = df_train[columns].values
    X = StandardScaler().fit_transform(X)
    df_norm = pd.DataFrame(X, columns=columns)
    df_corr = df_norm.corr()

    black_lst = []
    group = {}
    for col in columns:
        if col in black_lst:
            continue
        group[col] = list(df_corr[(df_corr[col]>=bcc_)|(df_corr[col]<=-bcc_)].index)
        black_lst +=  group[col]
    return group

def bubble_merge(df, columns=None, base_correlation_coefficient=.9, coverage_rate=.9):

    def is_similar(group1, group2):
        assert type(group1)==list, 'group1 should be a list'
        assert type(group2)==list, 'group2 should be a list'
        total_units = group1 + group2
        unique_units = list(set(total_units))
        common_parts = [col for col in unique_units if total_units.count(col)==2]
        if (len(common_parts)/len(group1) >= coverage_rate) | (len(common_parts)/len(group2) >= coverage_rate):
            return True
        else:
            return False

    def merge_group(original_group):
        group = original_group.copy()
        merged_group = group
        dict_list_ = list(group.items())
        is_merged = False

        index1 = 1
        for k1, v1 in dict_list_[:-1]:
            for k2,v2 in dict_list_[index1:]:
                    if is_similar(v1, v2):
                        group[k1] = list(set(v1 + v2))
                        del group[k2]
                        merged_group = merge_group(group)
                        is_merged = True
                        break
            if is_merged:
                break
            index1 += 1
        return merged_group

    if type(columns)==type(None):
        columns = df.columns.tolist()
    bcc_ = base_correlation_coefficient
    X = df[columns].values
    X = StandardScaler().fit_transform(X)
    df_norm = pd.DataFrame(X, columns=columns)
    df_corr = df_norm.corr()

    group = {}
    for col in columns:
        group[col] = list(df_corr[(df_corr[col]>=bcc_)|(df_corr[col]<=-bcc_)].index)

    return merge_group(group)